To do:
1.   Add the new questions
2.   Fix epsilon problem
3.   Let user add initial symptoms
4.   Separate men and women related questions (ask in the beginning)


# User Guide

Steps for training:


1.   Run all the hidden cells under the headline "Environment and Agent Codes" when you first open this document.
2.   Upload the weight folder by running the code under "Upload weights file". This is the file you'll start your training with, so check that you have the final version uploaded by the team. You can see the file is downloaded by clicking the small folder icon on your left.

3.   Run the code under the headline "Questions and Training code". You're now starting a training session, each session takes 5 cases.
4.   Write the full name of the weight file you just uploaded, example: weights_0000.hdf5, if no error shows, start training your cases.
5.   To continue training after your session ends, run the training code again, and write the name of the last weights file created on the left (usually called: weights_0004.hdf5). Run the code as many times as you wish.
6.    When you finish training. Before closing the document, right click and download the last weights file created on your left. (will probably be named the same "weights_0004.hdf5" since files will be overwritten).
7.    After making sure you downloaded the file, delete all weight files created on your left.
8.    Now please rename the downloaded weights file on your computer into this format: Yourname_Date.hdf5 (example: Nouran_18july.hdf5) and upload it to HE IT drive.






# Environment and Agent Codes:
Run only once when you open the code and don't change

In [ ]:
## Imports
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os


Using TensorFlow backend.


In [ ]:
## Environment

class healtius_env(object):
    def __init__(self, symptomsQuestions, diseases):
        """
        symptomsQuestions: 
            dictionary of questions mapped to symptoms.
            e.g. { "asthma" : "Have you been diagnosed with asthma?", "sweating_more" : "Are you sweating more than usual?"}
        diseases: 
            list of terminal states / diagnosis
            e.g. [diabetes, fever, ...]
        """
        self.symptomsQuestions = symptomsQuestions
        self.diseases = diseases

        self.possibleActions = list(self.symptomsQuestions.keys()) + diseases
        self.symptomState = np.zeros(len(self.symptomsQuestions.keys()))
        self.takenActions = set([])
        self.trueDiagnosis = 0

        self.stepcount = 0 # To keep track of number of questions asked

    def reset(self):
        """
        Resets the state of the environment and returns an initial observation.
        Returns:
            observation (object): the initial observation.
        """
        self.symptomState = np.zeros(len(self.symptomsQuestions.keys()))
        self.takenActions = set([])
        self.stepcount = 0
        print("\n########################## STATE RESET ##########################\n\n")
        self.trueDiagnosis = input("(Enter 1 for Anemia, 2 for Hyperthyroidism or 3 for Viral upper respiratory tract infection): ")
        return self.symptomState, self.trueDiagnosis

    
    def step(self, action):
        """
        Run one timestep of the environment's dynamics. When end of
        episode is reached, call `reset()`
        to reset this environment's state.
        Accepts an action and returns a tuple (observation/state, reward, done, info).
        Args:
            action (object): an action provided by the agent (An integer)
        Returns:
            observation/state (object): agent's observation of the current environment (state)
            reward (float) : amount of reward returned after previous action
            done (bool): whether the episode has ended, in which case further step() calls will return undefined results
            info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)
        """
        
        # Update State
        if action < len(self.symptomsQuestions): # action within "question space" / not ternimal actions
            answer = self.getAnswer(action)
            self.symptomState[action] = answer # update state

        # Calculate Reward
        done = False
        if action in self.takenActions: # repeated action
            reward = -1
            print("REPEATED QUESTION !! ")
            done = True
        elif len(self.takenActions) > 11:
            reward = -1
            print("Too many queations !")
            done = True
        elif action >= len(self.symptomsQuestions): # terminal action / terminal state
            if (action - len(self.symptomsQuestions) + 1) == int(self.trueDiagnosis):
              reward = 1
              print("DIAGNOSED CORRECTLY !! ",self.trueDiagnosis)
            else:
              reward = -1
              print("FALSE DIAGNOSIS !! ",self.trueDiagnosis,"instead got: ", (action - len(self.symptomsQuestions) + 1))
            done = True
        else: reward = 0

        # Add Action
        self.takenActions.add(action)

        self.stepcount +=1
        
        return self.symptomState, reward, done, None

    def getAnswer(self, action):
        """
        Arg:
            action is an integer
        Assumption:
            user response is an integer (1,-1) Future: magnitude to indicate severity?
        """
#         print(action)
        symptom = list(self.symptomsQuestions.keys())[action]
        while True:
          answer = input(self.symptomsQuestions[symptom]+"(Answer 1 or -1): ")
          if answer not in ("1","-1"):
            print("Invalid answer, please try again")
          else: break
        return answer

    def render(self):
        print('------------------------------------')
        print("Step: ", self.stepcount)
        print("Current state: ", self.symptomState)
        print("Actions taken: ", set(self.takenActions))
        print('------------------------------------')


In [ ]:
## Agent

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # double-ended queue; acts like list, but elements can be added/removed from either end
        self.gamma = 0.95 # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 1.0 # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.995 # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.01 # minimum amount of random exploration permitted
        self.learning_rate = 0.001 # rate at which NN adjusts models parameters via SGD to reduce cost 
        self.model = self._build_model() # private method 
    
    def _build_model(self):
        # neural net to approximate Q-value function:
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu')) # 1st hidden layer; states as input
        model.add(Dense(24, activation='relu')) # 2nd hidden layer
        model.add(Dense(self.action_size, activation='linear')) 
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later

    def act(self, state):
        if np.random.rand() <= self.epsilon: # if acting randomly, take random action
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # if not acting randomly, predict reward value based on current state
        return np.argmax(act_values[0]) # pick the action that will give the highest reward

    def replay(self, batch_size): # method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory, batch_size) # sample a minibatch from memory
        for state, action, reward, next_state, done in minibatch: # extract data for each minibatch sample
            target = reward # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            if not done: # if not done, then predict future discounted reward
                target = (reward + self.gamma * # (target) = reward + (discount rate gamma) * 
                          np.amax(self.model.predict(next_state)[0])) # (maximum target Q based on future action a')
            target_f = self.model.predict(state) # approximately map current state to future discounted reward
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)



# Upload Weights File Before Training

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


# Questions and Training Code

In [ ]:
## Questions and Training Code


if __name__ == '__main__':

    symptomsQuestions = { #3, 
                        "cough" : "Do you have a cough?",
                        "cough_color" : "Are you coughing up yellow, green or brown phlegm ?",
                        "cough_blood" : "Are you coughing up blood?",
                        "duration_hours" : "Have you had your complaints for hours ?",
                        "duration_days" : "Have you had your complaints for days ?",
                        "duration_weeks" : "Have you had your complaints for weeks ?",
                        "duration_months" : "Have you had your complaints for months ?",
                        "history_smoking" : "Do you smoke ?",
                        "history_allergy" : "Do you have a history of allergies, asthma,eczema ?",
                        "history_chestpain" : "Do you have any chest pain ?",
                        "history_immunity" : "Do you have a significantly weakned immunes system due to an existing condition or from taking medication ?",
                        "history_tired" : "Are you feeling tired, unwell, lethargic or run down ?",
                        "nose_blocked" : "Do you have blocked or stuffy nose ?",
                        "nose_runny" : "Do you have running or dripping nose ?",
                        "nose_sneeze" : "Do you have sneezing more than usual ?",
                        "nose_smell" : "Is  your sense of smell decreased ?",
                        "swallow_difficulty" : "Are you finding it difficult to swallow or having painful to swallow ?",                         
                        "swallow_solidfood" : "Are you having trouble swallowing solid food ?",                         
                        "swallow_fluids" : "Are you having trouble swallowing fluids ?",                         
                        "headache" : "Do you have a headache ?",                         
                        "headache_worst" : "Is the worst headache you can imagine ? ",
                        "headache_gradual" : "Did your headache start gradually ?",
                        "headache_sudden" : "Did your headache start very suddenly?",
                        "headache_discontinous" : "Does your headache come and go ?",
                        "headache_continous" : "Does your headache continue without interruption?",
                        "headache_neck" : "Do you have any problems moving your neck ?",
                        "headache_speech" : "Have you had any speech problems or noticed changes to your voice ?",
                        "headache_muscle" : "Have you noticed any abnormal muscle weakness lately ?",
                        "headache_vision" : "Have you noticed any change to your vision ?",
                        "headache_confused" : "Are you feeling confused or unable to remember things ?",                                     
                        "fever" : "Do you have a fever ?",                                     
                        "fever37" : "Is body temperature lower than  37 degree?",                                     
                        "fever38" : "Is body temperature between 37-38 degrees?",                                     
                        "fever40" : "Is body temperature between 38-40 degrees?",                                     
                        "fever41" : "Is body temperature greater than 40 degree?",                                     
                        "facepain" : "Do you have any pain in your face ?",                                     
                        "facepain_sinus" : "Is pain in your sinus ?",                                     
                        "musclepain" : "Do your muscles ache ?",
                        "jointpain" : "Do you have joint or bone pain in several areas of your body ?",                           
                        "throat" : "Do you have a sore throat ?",  
                        "throat_red" : "Do you have redness at the back of your throat?",  
                        "throat_tonsils" : "Do you have swollen tonsils?",  
                        "throat_white" : "Do you have white spots on your tonsils?",  
                        "throat_lump" : "Do you have lump in your throat?",  
                        "lung" : "Are you currently diagnosed with any of lung disorders?",  
                        "breathing" : "Have you experienced any difficulty breathing",  
                        "breathing_better" : "Is your difficulty breathing getting better ?",  
                        "breathing_same" : "Is your difficulty breathing staying the same ?",  
                        "breathing_worse" : "Is your difficulty breathing getting worse ?",  
                        "breathing_blue" : "Have your hands and feet gone blue",  
                        "breathing_sound" : "Have you noticed your breathing sounds wheezy or noisy",
                        #2, Hyperthyroidism
                        "palpitation_fast" : "Is your pulse racing faster than the normal limit for your age ?",
                        "palpitation_change" : "Do you feel your heart rhythm is change lately ?",
                        "weight_loss" : "Have you lost a significant amount of weight in recent weeks or months ?",
                        "weight_underweight" : "Are you underweight ?",
                        "weight_appetite" : "Have you lost your appetite ?",
                        "bowel_pain" : "Do you feel pain when having bowel movements?",
                        "bowel_blood" : "Does the diarrhoea contain blood?",
                        "bowel_largemotion" : "Have you been producing very large bowel motions?",
                        "bowel_diarrhoea" : "Do you have diarrhoea?",
                        "libido" : "Do you have loss of libido ?",
                        "paleness" : "Has your face become unusually pale?",
                        "menstrualcycle" : "Has your menstrual bleeding been occurring irregularly, i.e., at least 20 days between your shortest and longest menstrual cycle in the past six months?",
                        "muscleweakness_leg" : "Do you have weakness in any of the muscles in your leg or foot?",
                        "muscleweakness_movement" : "Are you unable to move the affected part of the body?",
                        "hair_thinner" : "Has your hair become thinner or sparser?",
                        "hair_loss" : "Have you noticed any recent hair loss on your scalp or face?",
                        "hair_break" : "Does your hair tend to easily break off just above your scalp?",
                        "hair_brows" : "Do you have lost any piece of your eyebrows?",
                        "nervous_aggressive" : "Are you acting aggressively recently?",
                        "nervous_mood" : "Do you have uncontrollable, large and quick changes in mood?",
                        "nervous_odd" : "Have you been told that you are behaving oddly?",
                        "heat_intolerance" : "Do you feel very uncomfortable in warm temperatures?",
                        "thirst" : "Do you feel extremely thirsty?",
                        "hands_tremble" : "Do your hands or another body part seem to tremble involuntarily?",
                        "insomnia" : "Do you have insomnia?",
                        "skin_moist" : "Is your skin moist more than ever?",
                        "skin_thinner" : "Is your skin thinner more than ever?",
                        "sweating" : "Are you sweating more than usual?",
                        "exophthalmos" : "Do you have swelling of the whole area around your eye?",
                        "breastenlargement_men" : "Do you have any lumps in your breast? (for men)",
                        "hyperactivity_exhaustion" : "Do you feel extremely exhausted?",
                        "hyperactivity_sleep" : "Have you noticed that you need less sleep recently?",
                        "hyperactivity_energy" : "Did you experience periods in the past with much higher mood and more energy than usual, and that lasted a week or longer?",
                        "hyperactivity_speech" : "Has your speech become unusually fast and hectic?",
                        "urine_high" : "Do you need to urinate more often than usual?",
                        #1, Anemia
                         
                        "headache_worse" : "Is it getting worse?",
                        "dizziness_past" : "Have you had similar episodes of dizziness or lightheadedness in the past?",
                        "dizziness" : "Do you feel dizzy?",
                        "dizziness_spinning" : "Do you feel dizzy, as if the room is spinning?",
                        "faster_breathing" : "Are you breathing faster than the normal limit for your age?",
                        "trouble_breathing" : "Have you trouble breathing when performing daily activities e.g., talking, sitting, or getting dressed?",
                        "chest_pain_pressure" : "Would you describe your chest pain as pressure?",
                        "family_heart_disease" : "Do your family have heart disease?",
                        "chest_pain_radiate" : "Is your chest pain radiating to the left arm or chin?",
                        "urine_low" : "Do you produce only a small amount of urine over the course of a day?",
                        "stools_blood" : "Has there been any bright red blood in or on your stools?",
                        "urine_dark" : "Has your urine become dark colored?",
                        "stools_black" : "Are your stools black or tarry?",
                        "enough_red_meat" : "Do you eat enough red meat?",
                        "vomit_brown" : "Did you vomit brown colorly liquid?",
                        "have_pica" : "Do you have PICA?",
                        "cold_hands_feet" : "Do your hand and feet feel cold?",
                        "nails_pale_brittle" : "Are your nails pale and brittle?",
                        "skin_pale" : "Is your skin pale?",
                        "sleepy_tired_weak_lackofenergy" : "Do you feel sleepy, drowsy, tired or weak, or do you lack the energy for your usual daily activities?",
                        "trouble_concentrating_remembering" : "Have you noticed troubles concentrating, remembering things, recognizing objects or learning new things?",
                        "less_alert_difficult_thinking" : "Are you feeling less alert, or having difficult thinking clearly?",
                        "lost_appetite" : "Have you lost your appetite?",
                        "feel_unrest" : "Do you feel unrest?",
                        "low_blood_pressure" : "Is your blood pressure lower than usual?",
                        "pulse_racing" : "Is your pulse racing faster than the normal limit for your age?",
                        "heart_rhythm_change" : "Do you feel your heart rhythm is change lately?",
                        "sleep_disturbance" : "Do you experience sleeping disturbances: sleep more or less than you used to, or you have trouble falling asleep or staying asleep?",
                        "b12_iron_deficiency_family" : "Do you have B12/iron deficiency in your family?",
                        "muscle_cramps" : "Have you been experiencing muscle cramps?",
                        "weaker_after_excercise" : "Do you feel weaker than usual after exercising?",
                        "muscle_aches_pains_soreness" : "Do you have muscle aches, pains or soreness?",
                        "tongue_red" : "Is your tongue unusually red?",
                        "tongue_sting_burning" : "Do you have a burning or stinging sensation on your tongue?",
                        "sklera_pale" : "Does your sklera appear paleness?",
                        "feel_cold_frequently" : "Do you get cold easily or feel cold frequently?",
                        "exposed_to_gas" : "Do you have, or are you near a coal or gas furnace, gas fireplace, or a broken gas boiler?",
                        "nosebleed" : "Have you had a nosebleed?",
                        "losing_more_hair" : "Are you losing more hair than usual?",
                        "use_pads_more_four" : "Do you use more than 3-4 pads daily?"
                     }
   
    diseases = ["Anemia",
                "Hyperthyroidism",
                "Viral upper respiratory tract infection"]
    state_size = len(symptomsQuestions.keys())
    action_size = len(symptomsQuestions.keys()) + len(diseases)
    batch_size = 32
    n_episodes = 5 # n times we want agent to train (default 1001)

    agent = DQNAgent(state_size, action_size) # initialise agent
    env = healtius_env(symptomsQuestions, diseases)

    # Load weights
    weights_file_name = input("Enter most recent model file name: ") 
    try:
      agent.load(weights_file_name) 
    except:
      print("Couldn't load model....")
      print("Will create a new model at the end of this code...")

    done = False

    
    for e in range(n_episodes): # iterate over new episodes of the game
        [state,_] = env.reset() # reset state at start of each new episode of the game
        state = np.reshape(state, [1, state_size])

        for time in range(100):  # do we need this? 
            env.render()
            action = agent.act(state) # agent picks an action
            next_state, reward, done, _ = env.step(action) # agent interacts with env, gets feedback       
            #reward = reward if not done else 10        
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.        
            state = next_state # set "current state" for upcoming iteration to the current next state        
            if done: # episode ends if agent drops pole or we reach timestep 5000
                print("episode: {}/{}, steps: {}, e: {:.2}" # print the episode's score and agent's epsilon
                    .format(e, n_episodes, time, agent.epsilon))
                break # exit loop
        if len(agent.memory) > batch_size:
            agent.replay(batch_size) # train the agent by replaying the experiences of the episode

        agent.save("weights_" + '{:04d}'.format(e) + ".hdf5")